In [178]:
import cv2 as cv
import numpy as np

In [179]:
number_list = [cv.imread("0.jpg", 0),
			   cv.imread("1.jpg", 0),
			   cv.imread("2.jpg", 0),
			   cv.imread("3.jpg", 0),
			   cv.imread("4.jpg", 0),
			   cv.imread("5.jpg", 0),
			   cv.imread("6.jpg", 0),
			   cv.imread("7.jpg", 0),
			   cv.imread("8.jpg", 0),
			   cv.imread("9.jpg", 0)]

image = cv.imread('match2.png')
imgGray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
template = cv.imread('win_label.jpg', 0)
colon_template = cv.imread('colon.jpg', 0)

In [180]:
from functools import cmp_to_key


time_region_width = 40


class AdvancedNumber:
	def __init__(self, number, box, isColon=False):
		self.number = number
		self.box = box
		self.isColon = isColon


def findTemplateRegion(img, template):
	w = template.shape[::-1][0]
	h = template.shape[::-1][1]
	res = cv.matchTemplate(img, template, cv.TM_CCOEFF_NORMED)
	_, _, _, max_loc = cv.minMaxLoc(res)
	return max_loc, (max_loc[0] + w, max_loc[1] + h), res


def findTemplateBoxes(img, template, threshold=0.9):
	w = template.shape[::-1][0]
	h = template.shape[::-1][1]
	res = cv.matchTemplate(img, template, cv.TM_CCOEFF_NORMED)
	detected_boxes = []
	loc = np.where(res >= threshold)
	for pt in zip(*loc[::-1]):
		detected_boxes.append((pt, (pt[0] + w, pt[1] + h)))
	return detected_boxes


def findCenter(box):
	return (box[0][0] + box[1][0]) * 0.5


def comparator(item1, item2):
	if findCenter(item1.box) < findCenter(item2.box):
		return -1
	elif findCenter(item1.box) > findCenter(item2.box):
		return 1
	else:
		return 0

In [181]:
top_left, bottom_right, _ = findTemplateRegion(imgGray, template)
time_region = imgGray[top_left[1]: bottom_right[1], bottom_right[0]: bottom_right[0] + time_region_width]

In [182]:
founded_number = []
for idx, number in enumerate(number_list):
	boxes = findTemplateBoxes(time_region, number)
	for box in boxes:
		founded_number.append(AdvancedNumber(idx, box))

colons = findTemplateBoxes(time_region, colon_template)
for colon in colons:
	founded_number.append(AdvancedNumber(-1, colon, True))

founded_number = sorted(founded_number, key=cmp_to_key(comparator))
time = ''
for number in founded_number:
	if number.isColon:
		time += ':'
	else:
		time += str(number.number)

print(time)

38:49
